In [ ]:
import numpy as np
from scipy.sparse.linalg import eigs, LinearOperator
from ncon import ncon
from tutorialFunctions import createMPS, mixedCanonical, minAcC
from time import time
import matplotlib.pyplot as plt

## 3. Two-dimensional partition functions using VUMPS

We can also construct the partition function of a two-dimensional Hamiltonian by considering for example the 2D Ising Hamiltonian. This is done by constructing a one-dimensional transfer matrix in the form of a matrix product operator (MPO), and noting that the partition function is then an (infinite) product of these transfer matrices, stacked vertically.

![transfer matrix](img/transferMPO.png)

As we will be working in the thermodynamic limit, it is again clear that we will be needing the leading eigenvectors of these transfer matrices, the fixed points. We also assume that the transfer matrix is properly normalised, such that the leading eigenvalue is 1.

### 2.1. Tensors

We start by defining the tensors that make up the transfer matrix for the Ising model. If we consider an infinite square lattice, we may simulate the model by placing a spin $1/2$ degree of freedom on each vertex, and then characterising the interactions by inserting a matrix $Q$ with Boltzmann weights on each of the edges. In the graphical language of tensor networks this corresponds to having a 4-legged 'Dirac delta tensor' on the vertices and a 2-legged $Q$ on the bonds.

If we want to rephrase this in terms of the transfer matrix we defined above, we split every bond interaction matrix over the two adjacent vertices and absorp them in the definition of $O$.

In [ ]:
def isingO(beta, J):
    """
    Gives the MPO tensor corresponding to the partition function of the 2d 
    classical Ising model at a given temperature and coupling, obtained by
    distributing the Boltzmann weights evenly over all vertices.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        O : np.array (2, 2, 2, 2)
            MPO tensor,
            ordered left-top-right-bottom.

    """
    # basic vertex tensor
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    # build square root of matrix of Boltzmann weights and pull into vertex edges
    c, s = np.sqrt(np.cosh(beta*J)), np.sqrt(np.sinh(beta*J))
    Qsqrt = 1/np.sqrt(2) * np.array([[c+s, c-s],[c-s, c+s]])
    O = ncon((Qsqrt, Qsqrt, Qsqrt, Qsqrt, vertex), ([-1,1], [-2,2], [-3,3], [-4,4], [1,2,3,4]))
    return O

def isingM(beta, J):
    """
    Gives the magnetizatopn MPO tensor for the 2d classical Ising model at a
    given temperature and coupling.
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        M : np.array (2, 2, 2, 2)
            Magnetization MPO tensor,
            ordered left-top-right-bottom.

    """
    vertex = np.zeros( (2,) * 4 )
    vertex[tuple([np.arange(2)] * 4)] = 1
    Z = np.array([[1,0],[0,-1]])
    c, s = np.sqrt(np.cosh(beta*J)), np.sqrt(np.sinh(beta*J))
    Qsqrt = 1/np.sqrt(2) * np.array([[c+s, c-s],[c-s, c+s]])
    vertexZ = ncon((Z, vertex), ([-1,1], [1,-2,-3,-4]))
    M = ncon((Qsqrt, Qsqrt, Qsqrt, Qsqrt, vertexZ), ([-1,1], [-2,2], [-3,3], [-4,4], [1,2,3,4]))
    return M

### 2.2 MPS as an MPO fixed point
The partition function is given by contracting a tensor network consisting of all these rank 4 MPO tensors $O$, which have dimensions $(d,d,d,d)$. Whenever we contract such an infinite two-dimensional tensor network, we want to find the fixed point of the transfer matrix, such that we can collapse the vertical direction of the diagram.

Indeed, having found an MPS representation $|\Psi(A)>$ of the fixed point of $T(O)$, the partition function is evaluated to be

$$ \Lambda = <\Psi(A)| T |\Psi(A) > $$

Bringing the MPS in mixed canonical form, we write

![partition Function](img/partFunc.img)

### 2.3 Left and right fixed points
Again, contracting this infinite network requires that we find $F_L$ and $F_R$, such that we may now collapse the horizontal direction. These are found by considering the following diagrams:

![FL](img/FL.png)
![FR](img/FR.png)

Finally these are properly normalised if we set
![normal](img/FlFr.png)


In [ ]:
def leftFixedPointMPO(O, Al, tol):
    """
    Computes the left fixed point (250).

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Al : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            left-orthonormal.
        tol : float, optional
            current tolerance

        Returns
        -------
        lam : float
            Leading left eigenvalue.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.

    """
    D = Al.shape[0]
    d = Al.shape[1]
    
    # construct handle for the action of the relevant operator and cast to linear operator
    transferLeftHandleMPO = lambda v: (ncon((v.reshape((D,d,D)), Al, np.conj(Al), O),([5, 3, 1], [1, 2, -3], [5, 4, -1], [3, 2, -2, 4]))).reshape(-1)
    transferLeftMPO = LinearOperator((D**2*d, D**2*d), matvec=transferLeftHandleMPO)
    lam, Fl = eigs(transferLeftMPO, k=1, which="LM", tol=tol)
    
    return lam, Fl.reshape((D,d,D))


def rightFixedPointMPO(O, Ar, tol):
    """
    Computes the right fixed point (250).

        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ar : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            right-orthonormal.
        tol : float, optional
            current tolerance

        Returns
        -------
        lam : float
            Leading right eigenvalue.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.

    """
    D = Ar.shape[0]
    d = Ar.shape[1]
    
    # construct handle for the action of the relevant operator and cast to linear operator
    transferRightHandleMPO = lambda v: (ncon((v.reshape(D, d, D), Ar, np.conj(Ar), O), ([1, 3, 5], [-1, 2, 1], [-3, 4, 5], [-2, 2, 3, 4]))).reshape(-1)
    transferRightMPO = LinearOperator((D**2*d, D**2*d), matvec=transferRightHandleMPO)
    lam, Fr = eigs(transferRightMPO, k=1, which="LM", tol=tol)
    
    return lam, Fr.reshape((D,d,D))

def overlapFixedPointsMPO(Fl, Fr, C):
    """
    Performs the contraction that gives the overlap of the fixed points (251).

        Parameters
        ----------
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.

        Returns
        -------
        overlap : float
            Overlap of the fixed points.

    """
    
    overlap = ncon((Fl, Fr, C, np.conj(C)), ([1, 3, 2], [5, 3, 4], [2, 5], [1, 4]))
    
    return overlap


### 2.4 Optimize overlap using the VUMPS algorithm

We now want to reuse the VUMPS algorithm for optimizing the fixed point we used in the vertical direction, which is represented by an MPS.

To do so, we calculate the tangent-space projector and again find two terms that can be characterised by the maps

![OAc](img/OAc.png)
![OC](img/OC.png)

Again together with the consistency conditions, a fixed point is thus characterised by the set of equations:

$$ O_{A_C}(A_C) \sim A_C $$
$$ O_C(C) \sim C $$
$$ A_C = A_L C = CA_R $$

In [ ]:
def O_Ac(X, O, Fl, Fr, lam):
    """
    Action of the map (256) on a given tensor.

        Parameters
        ----------
        X : np.array(D, d, D)
            Tensor of size (D, d, D)
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.

        Returns
        -------
        Xnew : np.array(D, d, D)
            Result of the action of O_Ac on the tensor X.

    """
    
    Xnew = ncon((Fl, Fr, X, O),([-1, 2, 1], [4, 5, -3], [1, 3, 4], [2, 3, 5, -2])) / lam
    
    return Xnew


def O_C(X, Fl, Fr):
    """
    Action of the map (257) on a given tensor.

        Parameters
        ----------
        X : np.array(D, D)
            Tensor of size (D, D)
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.

        Returns
        -------
        Xnew : np.array(D, d, D)
            Result of the action of O_C on the tensor X.

    """
    
    Xnew = ncon((Fl, Fr, X), ([-1, 3, 1], [2, 3, -2], [1, 2]))
    
    return Xnew

Solving the eigenvalue equations gives us a new $\tilde{A}_C$ and $\tilde{C}$:

In [ ]:
def calcNewCenterMPO(O, Ac, C, Fl, Fr, lam, tol=1e-3):
    """
    Find new guess for Ac and C as fixed points of the maps O_Ac and O_C.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        lam : float
            Leading eigenvalue.
        tol : float, optional
            current tolerance
    
        Returns
        -------
        AcTilde : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            center gauge.
        CTilde : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.

    """
    
    D = Ac.shape[0]
    d = Ac.shape[1]
    
    # construct handle for O_Ac map and cast to linear operator
    handleAc = lambda X: (O_Ac(X.reshape((D,d,D)), O, Fl, Fr, lam)).reshape(-1)
    handleAc = LinearOperator((D**2*d, D**2*d), matvec=handleAc)
    # construct handle for O_C map and cast to linear operator
    handleC = lambda X: (O_C(X.reshape(D, D), Fl, Fr)).reshape(-1)
    handleC = LinearOperator((D**2, D**2), matvec=handleC)
    # compute fixed points of these maps: gives new guess for center tensors
    _, AcTilde = eigs(handleAc, k=1, which="LM", v0=Ac.reshape(-1), tol=tol)
    _, CTilde = eigs(handleC, k=1, which="LM", v0=C.reshape(-1), tol=tol)
    
    # reshape to tensors of correct size
    AcTilde = AcTilde.reshape((D, d, D))
    CTilde = CTilde.reshape((D, D))
    
    return AcTilde, CTilde

And after retrieving $\tilde{A}_C$ and $\tilde{C}$ we can recycle the same 'minAcC()' function to iterate the whole procedure until convergence. The partition function is eventually given by the left and right fixed point eigenvalue lambda. To check this we define some functions calculating the free energy and the magnetization for the simulation and the exact solution.

In [ ]:
def isingMagnetization(beta, J, Ac, Fl, Fr):
    """
    Computes the expectation value of the magnetization in the Ising model
    for a given temperature and coupling
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
        Ac : np.array(D, d, D)
            MPS tensor of the MPS fixed point,
            with 3 legs ordered left-bottom-right,
            center gauge.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
    
        Returns
        -------
        M : float
            Expectation value of the magnetization at the given temperature
            and coupling.

    """
    return ncon((Fl, Ac, isingM(beta, J), np.conj(Ac), Fr), (
        [1, 3, 2], [2,7,5],[3,7,8,6],[1,6,4], [5,8,4]))


def isingZ(beta, J, Ac, Fl, Fr):
    """
    Computes the Ising model partition function for a given temperature and
    coupling
    
        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
        Ac : np.array(D, d, D)
            MPS tensor of the MPS fixed point,
            with 3 legs ordered left-bottom-right,
            center gauge.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
    
        Returns
        -------
        Z : float
            Value of the partition function at the given temperature and
            coupling.

    """
    
    Z = ncon((Fl, Ac, isingO(beta, J), np.conj(Ac), Fr), (
        [1, 3, 2], [2,7,5],[3,7,8,6],[1,6,4], [5,8,4]))
    
    return Z

def isingExact(beta, J):
    """
    Exact Onsager solution for the 2d classical Ising Model

        Parameters
        ----------
        beta : float
            Inverse temperature.
        J : float
            Coupling strength.
    
        Returns
        -------
        magnetization : float
            Magnetization at given temperature and coupling.
        free : float
            Free energy at given temperature and coupling.
        energy : float
            Energy at given temperature and coupling.

    """
    theta = np.arange(0, np.pi/2, 1e-6)
    x = 2 * np.sinh(2 * J * beta) / np.cosh(2 * J * beta) ** 2
    if 1 - (np.sinh(2 * J * beta)) ** (-4) > 0:
        magnetization = (1 - (np.sinh(2 * J * beta)) ** (-4)) ** (1 / 8)
    else:
        magnetization = 0
    free = -1 / beta * (np.log(2 * np.cosh(2 * J * beta)) + 1 / np.pi * np.trapz(np.log(1 / 2 * (1 + np.sqrt(1 - x ** 2 * np.sin(theta) ** 2))), theta))
    K = np.trapz(1 / np.sqrt(1 - x ** 2 * np.sin(theta) ** 2), theta)
    energy = -J * np.cosh(2 * J * beta) / np.sinh(2 * J * beta) * (1 + 2 / np.pi * (2 * np.tanh(2 * J * beta) ** 2 - 1) * K)
    return magnetization, free, energy

Calculating the partition function then becomes an iterative procedure.

In [ ]:
def vumpsMPO(O, D, A0=None, tol=1e-4):
    """
    Find the fixed point MPS of a given MPO using VUMPS.
    
        Parameters
        ----------
        O : np.array (d, d, d, d)
            MPO tensor,
            ordered left-top-right-bottom.
        D : int
            Bond dimension
        A0 : np.array (D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            initial guess.
        tol : float
            Relative convergence criterium.
        
        Returns
        -------
        lam : float
            Leading eigenvalue.
        Al : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            left orthonormal.
        Ar : np.array(D, d, D)
            MPS tensor zith 3 legs,
            ordered left-bottom-right,
            right orthonormal.
        Ac : np.array(D, d, D)
            MPS tensor with 3 legs,
            ordered left-bottom-right,
            center gauge.
        C : np.array(D, D)
            Center gauge with 2 legs,
            ordered left-right.
        Fl : np.array(D, d, D)
            left fixed point,
            ordered bottom-middle-top.
        Fr : np.array(D, d, D)
            right fixed point,
            ordered top-middle-bottom.
        
    """
    
    d = O.shape[0]
    
    # if no initial guess, random one
    if A0 is None:
        A0 = createMPS(D, d)
    
    # go to mixed gauge
    Al, Ac, Ar, C = mixedCanonical(A0)
    
    delta = 1e-4
    flag = True
    while flag:
        # compute left and right fixed points
        lam, Fl = leftFixedPointMPO(O, Al, delta/10)
        _ , Fr = rightFixedPointMPO(O, Ar, delta/10)
        Fl /= overlapFixedPointsMPO(Fl, Fr, C)
        lam = np.real(lam)[0]
        # compute updates on Ac and C
        AcTilde, CTilde = calcNewCenterMPO(O, Ac, C, Fl, Fr, lam, delta/10)
        AlTilde, AcTilde, ArTilde, CTilde = minAcC(AcTilde, CTilde)
        # calculate convergence measure, check for convergence
        delta = np.linalg.norm(O_Ac(Ac, O, Fl, Fr, lam) - ncon((Al, O_C(C, Fl, Fr)), ([-1, -2, 1], [1, -3])))
        if delta < tol:
            flag = False
        # update tensors
        Al, Ac, Ar, C = AlTilde, AcTilde, ArTilde, CTilde
    
    return lam, Al, Ac, Ar, C, Fl, Fr

Using these functionalities, we can now compute the magnetization and the free energy as a function of the temperature:

In [ ]:
D = 12
d = 2
J = 1

print('Bond dimension: D =', D)
Al = createMPS(D, d)
# optimization parameters
tol = 1e-5

Ts = np.linspace(1., 3.4, 100)
magnetizations = []
magnetizationsExact = []
freeEnergies = []
freeEnergiesExact = []

for T in Ts:
    beta = 1/T
    O = isingO(beta, J)
    print('T={}'.format(T))
    
    lam, Al, Ac, Ar, C, Fl, Fr = vumpsMPO(O, D, A0=Al, tol=tol)
    magnetizations.append(np.abs(isingMagnetization(beta, J, Ac, Fl, Fr)/isingZ(beta, J, Ac, Fl, Fr)))
    magnetizationsExact.append(isingExact(beta, J)[0])
    freeEnergies.append(-np.log(lam)/beta)
    freeEnergiesExact.append(isingExact(beta, J)[1])


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5), dpi=120)
ax1.set_title('Magnetization as a function of the temperature')
ax1.set(xlabel=r'$T$', ylabel=r'$<M>$')
ax1.scatter(Ts, magnetizations, label = 'D={}'.format(D), c="hotpink", marker="^")
ax1.plot(Ts, magnetizationsExact, label = 'exact')

ax2.set_title('Free energy as a function of the temperature')
ax2.set(xlabel=r'$T$', ylabel=r'$f$')
ax2.scatter(Ts, freeEnergies, label = 'D={}'.format(D), c="hotpink", marker="^")
ax2.plot(Ts, freeEnergiesExact, label = 'exact')
plt.legend()
